In [1]:
import pandas as pd
import os
from PIL import Image

In [2]:
df = pd.read_json('../training_data/seeclick_web.json')
df.head(1)

,img_filename,url,elements
0,e4469ac844e851b42167f303c8c9278e.png,https://lunaweddingandeventsupplies.com.au/pro...,"[{'instruction': 'Product description', 'bbox'..."


In [3]:
df.columns, df.dtypes

(Index(['img_filename', 'url', 'elements'], dtype='object'),
 img_filename    object
 url             object
 elements        object
 dtype: object)

In [4]:
image_names = set(os.listdir('../training_data/cpfs01/user/chengkanzhi/seeclick_web_imgs_part/'))
df = df[df['img_filename'].isin(image_names)]
len(df)

10000

In [5]:
print(len(df))
df = df.explode('elements').reset_index(drop=True)

df['instruction'] = df['elements'].apply(lambda x: x['instruction'])
df['bbox'] = df['elements'].apply(lambda x: x['bbox'])

10000


In [7]:
df.drop(columns=['url', 'elements'], inplace=True)
df.head(2)

,img_filename,instruction,bbox
0,b1072ebd95d94a714a204ad0a44c2cb0.png,Composers -- Western Australia.,"[0.1125, 0.810185185185185, 0.202604166666666,..."
1,b1072ebd95d94a714a204ad0a44c2cb0.png,InterLibrary Loan,"[0.7187500000000001, 0.28518518518518504, 0.76..."


In [8]:
len(df)

113142

In [ ]:
def get_quadrant(image, quadrant_idx):
    width, height = image.size
    if quadrant_idx == 0:  # Top-left
        return image.crop((0, 0, width // 2, height // 2))
    elif quadrant_idx == 1:  # Top-right
        return image.crop((width // 2, 0, width, height // 2))
    elif quadrant_idx == 2:  # Bottom-left
        return image.crop((0, height // 2, width // 2, height))
    elif quadrant_idx == 3:  # Bottom-right
        return image.crop((width // 2, height // 2, width, height))

In [9]:
def create_new_points(row, k=4):
    img = Image.open(f'../training_data/cpfs01/user/chengkanzhi/seeclick_web_imgs_part/{row["img_filename"]}')
    w, h = img.size
    bbox = row['bbox']
    instruction = row['instruction']
    file_name = row['img_filename']
    x1, y1, x2, y2 = bbox
    x, y = (x1 + x2) / 2, (y1 + y2) / 2
    x, y = x*w, y*h
    return_val = []
    img_crop = [0, 0, w, h]
    for i in range(1, k):
        w, h = img.size
        mid_w, mid_h = w / 2, h / 2
        new_x, new_y = x, y
        new_crop = img_crop
        # Determine the quadrant and adjust the bounds
        if x < mid_w and y < mid_h:  # Quadrant 1
            quadrant = 1
            new_crop = (0, 0, mid_w, mid_h)
        elif x >= mid_w and y < mid_h:  # Quadrant 2
            quadrant = 2
            new_crop = (mid_w, 0, w, mid_h)
            new_x -= mid_w  # Adjust x relative to the new image bounds
        elif x < mid_w and y >= mid_h:  # Quadrant 3
            quadrant = 3
            new_crop = (0, mid_h, mid_w, h)
            new_y -= mid_h  # Adjust y relative to the new image bounds
        else:  # x >= mid_w and y >= mid_h, Quadrant 4
            quadrant = 4
            new_crop = (mid_w, mid_h, w, h)
            new_x -= mid_w  # Adjust x relative to the new image bounds
            new_y -= mid_h  # Adjust y relative to the new image bounds
    
        new_point = (file_name, instruction, img_crop, quadrant, (x, y), i)
        return_val.append(new_point)
        img_crop = new_crop
        img = img.crop(new_crop)
        x, y = new_x, new_y

    return return_val



new_points = []
for _, row in df.iterrows():
    new_points.extend(create_new_points(row))
    

df = pd.DataFrame(new_points, columns=['img_filename', 'instruction', 'crop', 'quadrant', 'point', 'k'])
df.head(10)

,img_filename,instruction,crop,quadrant,point,k
0,b1072ebd95d94a714a204ad0a44c2cb0.png,Composers -- Western Australia.,"[0, 0, 1920, 1080]",1,"(0.157552083333333, 0.8166666666666665)",1
1,b1072ebd95d94a714a204ad0a44c2cb0.png,Composers -- Western Australia.,"(0, 0, 960.0, 540.0)",1,"(0.157552083333333, 0.8166666666666665)",2
2,b1072ebd95d94a714a204ad0a44c2cb0.png,Composers -- Western Australia.,"(0, 0, 960.0, 540.0)",1,"(0.157552083333333, 0.8166666666666665)",3
3,b1072ebd95d94a714a204ad0a44c2cb0.png,InterLibrary Loan,"[0, 0, 1920, 1080]",1,"(0.74140625, 0.2907407407407405)",1
4,b1072ebd95d94a714a204ad0a44c2cb0.png,InterLibrary Loan,"(0, 0, 960.0, 540.0)",1,"(0.74140625, 0.2907407407407405)",2
5,b1072ebd95d94a714a204ad0a44c2cb0.png,InterLibrary Loan,"(0, 0, 960.0, 540.0)",1,"(0.74140625, 0.2907407407407405)",3
6,b1072ebd95d94a714a204ad0a44c2cb0.png,Bookmark link for this record,"[0, 0, 1920, 1080]",1,"(0.5882812499999995, 0.30000000000000004)",1
7,b1072ebd95d94a714a204ad0a44c2cb0.png,Bookmark link for this record,"(0, 0, 960.0, 540.0)",1,"(0.5882812499999995, 0.30000000000000004)",2
8,b1072ebd95d94a714a204ad0a44c2cb0.png,Bookmark link for this record,"(0, 0, 960.0, 540.0)",1,"(0.5882812499999995, 0.30000000000000004)",3
9,b1072ebd95d94a714a204ad0a44c2cb0.png,Author,"[0, 0, 1920, 1080]",1,"(0.889583333333333, 0.2240740740740735)",1


In [ ]:
df.to_parquet(
    'data.parquet', snappy=True
)